# Homework: Phrasal Chunking

This is your documentation for the chunker homework.

### For epochs = 10

In [4]:
%%time
import perc
import default
import sys
from collections import defaultdict

feat_vec = {}
tagset = []
train_data = []

def perc_train(train_data, tagset, numepochs):
    count = 0 
    feat_vec = defaultdict(int)
    new_feat_vec = defaultdict(float)
    for epoch in range(numepochs):
        print("Epoch #:", epoch)
        positives = 0
        negatives = 0
        for data in train_data:
            words = []
            chunks = []
            text = data[0] #in IN B-PP
            feat = data[1] #U01:_B-1
            for info in text:
                (word, pos, chunk) = info.split()
                words.append(word) 
                chunks.append(chunk)
            tag0 = tagset[0]
            # Get the output tags for the input sentence
            output_tags = perc.perc_test(feat_vec=feat_vec, 
                                         labeled_list=text, 
                                         feat_list=feat, 
                                         tagset=tagset, 
                                         default_tag=tag0)
            feat_index = 0
            counter = 0
            #If chunking tag is incorrect, decrease the feature vector weight by 1; Increase negative count by 1
            #Increase the weight by 1 for every correct tag. Increment positive counter by 1 
            for word in words:
                (feat_index, feats) = perc.feats_for_word(feat_index, feat_list=feat)
                tag = output_tags[counter]
                best_tag = chunks[counter]
                if (tag == best_tag):
                    counter +=1
                    continue
                for ft in feats:
                    correct = ft, best_tag
                    incorrect = ft, tag
                    feat_vec[correct] += 1
                    feat_vec[incorrect] -= 1
                counter += 1
            counter = 0
            
            for word in words:
                tag = output_tags[counter]
                best_tag = chunks[counter]
                if (tag == best_tag):
                    counter += 1
                    positives += 1
                    continue
                else:
                    negatives += 1

                prev_value = "B:"
                prev_tag = "B:"
                if (counter == 0):
                    prev_value += "_B-1"
                    prev_tag += "_B-1"   
                else:
                    prev_value += chunks[counter - 1]
                    prev_tag += output_tags[counter - 1]
                correct = prev_value, best_tag
                incorrect = prev_tag, tag
                feat_vec[correct] += 1
                feat_vec[incorrect] -= 1
                counter += 1

            for key in feat_vec.keys():
                new_feat_vec[key] += feat_vec[key]
            count+=1

        print("Correctly identified:", positives)
        print("Number of mistakes:", negatives)

    for key in new_feat_vec.keys():
        new_feat_vec[key] = new_feat_vec[key]/float(count)
    return new_feat_vec


tagset = perc.read_tagset("data/tagset.txt")
print("reading data ...", file=sys.stderr)
train_data = perc.read_labeled_data("data/train.txt.gz", "data/train.feats.gz", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc_train(train_data, tagset, 10)
perc.perc_write_to_file(feat_vec, "default.model")
print("wrote model to disk")

reading data ...
done.


Epoch #: 0
Correctly identified: 195245
Number of mistakes: 16482
Epoch #: 1
Correctly identified: 202691
Number of mistakes: 9036
Epoch #: 2
Correctly identified: 205586
Number of mistakes: 6141
Epoch #: 3
Correctly identified: 207445
Number of mistakes: 4282
Epoch #: 4
Correctly identified: 208458
Number of mistakes: 3269
Epoch #: 5
Correctly identified: 209159
Number of mistakes: 2568
Epoch #: 6
Correctly identified: 209622
Number of mistakes: 2105
Epoch #: 7
Correctly identified: 210159
Number of mistakes: 1568
Epoch #: 8
Correctly identified: 210292
Number of mistakes: 1435
Epoch #: 9
Correctly identified: 210554
Number of mistakes: 1173
wrote model to disk
CPU times: user 3h 15min 46s, sys: 46.1 s, total: 3h 16min 32s
Wall time: 6h 41min 7s


In [7]:
%%capture --no-stderr output
print("reading test data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc.perc_read_from_file("default.model")
perc.perc_testall(feat_vec, test_data, tagset)

reading test data ...
done.


In [8]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(str(output), boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 5790; correct phrases: 5407
             ADJP: precision:  73.47%; recall:  72.73%; F1:  73.10; found:     98; correct:     99
             ADVP: precision:  77.29%; recall:  79.21%; F1:  78.24; found:    207; correct:    202
            CONJP: precision: 100.00%; recall:  60.00%; F1:  75.00; found:      3; correct:      5
             INTJ: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      1
               NP: precision:  94.24%; recall:  94.02%; F1:  94.13; found:   3019; correct:   3026
               PP: precision:  96.92%; recall:  97.95%; F1:  97.43; found:   1234; correct:   1221
              PRT: precision:  80.00%; recall:  72.73%; F1:  76.19; found:     20; correct:     22
             SBAR: precision:  84.76%; recall:  83.18%; F1:  83.96; found:    105; correct:    107
               VP: precision:  92.93%; recall:  93.27%; F1:  93.10; found:   1104; correct:   1100
accura

### For epochs = 5

In [1]:
%%time
import perc
import default
import sys
from collections import defaultdict

feat_vec = {}
tagset = []
train_data = []

def perc_train(train_data, tagset, numepochs):
    count = 0 
    feat_vec = defaultdict(int)
    new_feat_vec = defaultdict(float)
    for epoch in range(numepochs):
        print("Epoch #:", epoch)
        positives = 0
        negatives = 0
        for data in train_data:
            words = []
            chunks = []
            text = data[0] #in IN B-PP
            feat = data[1] #U01:_B-1
            for info in text:
                (word, pos, chunk) = info.split()
                words.append(word) 
                chunks.append(chunk)
            tag0 = tagset[0]
            
            output_tags = perc.perc_test(feat_vec=feat_vec, 
                                         labeled_list=text, 
                                         feat_list=feat, 
                                         tagset=tagset, 
                                         default_tag=tag0)
            feat_index = 0
            counter = 0

            for word in words:
                (feat_index, feats) = perc.feats_for_word(feat_index, feat_list=feat)
                tag = output_tags[counter]
                best_tag = chunks[counter]
                if (tag == best_tag):
                    counter +=1
                    continue
                for ft in feats:
                    correct = ft, best_tag
                    incorrect = ft, tag
                    feat_vec[correct] += 1
                    feat_vec[incorrect] -= 1
                counter += 1
            counter = 0
            
            for word in words:
                tag = output_tags[counter]
                best_tag = chunks[counter]
                if (tag == best_tag):
                    counter += 1
                    positives += 1
                    continue
                else:
                    negatives += 1

                prev_value = "B:"
                prev_tag = "B:"
                if (counter == 0):
                    prev_value += "_B-1"
                    prev_tag += "_B-1"   
                else:
                    prev_value += chunks[counter - 1]
                    prev_tag += output_tags[counter - 1]
                correct = prev_value, best_tag
                incorrect = prev_tag, tag
                feat_vec[correct] += 1
                feat_vec[incorrect] -= 1
                counter += 1

            for key in feat_vec.keys():
                new_feat_vec[key] += feat_vec[key]
            count+=1

        print("Correctly identified:", positives)
        print("Number of mistakes:", negatives)

    for key in new_feat_vec.keys():
        new_feat_vec[key] = new_feat_vec[key]/float(count)
    return new_feat_vec


tagset = perc.read_tagset("data/tagset.txt")
print("reading data ...", file=sys.stderr)
train_data = perc.read_labeled_data("data/train.txt.gz", "data/train.feats.gz", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc_train(train_data, tagset, 5)
perc.perc_write_to_file(feat_vec, "default.model")
print("wrote model to disk")

reading data ...
done.


Epoch #: 0
Correctly identified: 195245
Number of mistakes: 16482
Epoch #: 1
Correctly identified: 202691
Number of mistakes: 9036
Epoch #: 2
Correctly identified: 205586
Number of mistakes: 6141
Epoch #: 3
Correctly identified: 207445
Number of mistakes: 4282
Epoch #: 4
Correctly identified: 208458
Number of mistakes: 3269
wrote model to disk
CPU times: user 1h 23min 5s, sys: 24.5 s, total: 1h 23min 29s
Wall time: 1h 24min 15s


In [2]:
%%capture --no-stderr output
print("reading test data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc.perc_read_from_file("default.model")
perc.perc_testall(feat_vec, test_data, tagset)

reading test data ...
done.


In [3]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(str(output), boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 5796; correct phrases: 5399
             ADJP: precision:  71.13%; recall:  69.70%; F1:  70.41; found:     97; correct:     99
             ADVP: precision:  76.70%; recall:  78.22%; F1:  77.45; found:    206; correct:    202
            CONJP: precision: 100.00%; recall:  60.00%; F1:  75.00; found:      3; correct:      5
             INTJ: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      1
               NP: precision:  94.08%; recall:  93.99%; F1:  94.03; found:   3023; correct:   3026
               PP: precision:  96.93%; recall:  98.12%; F1:  97.52; found:   1236; correct:   1221
              PRT: precision:  75.00%; recall:  68.18%; F1:  71.43; found:     20; correct:     22
             SBAR: precision:  83.96%; recall:  83.18%; F1:  83.57; found:    106; correct:    107
               VP: precision:  92.58%; recall:  93.00%; F1:  92.79; found:   1105; correct:   1100
accura